In [9]:
%load_ext autoreload

from utils import BraketTaskScanner, DeviceScanner,DeviceUtils

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.device_schema.device_action_properties import DeviceActionType
import numpy as np



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#list gate we want to investigate

dev_scanner = DeviceScanner( device =DeviceUtils.get_device("ionq"))

print(dev_scanner.get_native_gates())
print(dev_scanner.get_supported_gates())

['GPI', 'GPI2', 'MS']
['x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cnot', 's', 'si', 't', 'ti', 'v', 'vi', 'xx', 'yy', 'zz', 'swap']


In [25]:
# ionq misterious native gates
# https://ionq.com/docs/getting-started-with-native-gates

gpi = Circuit().gpi(1,np.pi)
print(gpi.to_unitary())

print("###############")

gpi2 = Circuit().gpi2(1, np.pi)
print(gpi2.to_unitary())

print("###############")

ms = Circuit().ms(1,2, np.pi,0)
print(ms.to_unitary())

[[ 0.+0.0000000e+00j -1.-1.2246468e-16j]
 [-1.+1.2246468e-16j  0.+0.0000000e+00j]]
###############
[[ 7.07106781e-01+0.j         -8.65956056e-17+0.70710678j]
 [ 8.65956056e-17+0.70710678j  7.07106781e-01+0.j        ]]
###############
[[ 7.07106781e-01+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j         -8.65956056e-17+0.70710678j]
 [ 0.00000000e+00+0.j          7.07106781e-01+0.j
  -8.65956056e-17+0.70710678j  0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          8.65956056e-17+0.70710678j
   7.07106781e-01+0.j          0.00000000e+00+0.j        ]
 [ 8.65956056e-17+0.70710678j  0.00000000e+00+0.j
   0.00000000e+00+0.j          7.07106781e-01+0.j        ]]


Hadamard gate

In [2]:
#see how quistkid transpile a Hadamard in ionq supported gates

from qiskit import QuantumCircuit, QuantumRegister,ClassicalRegister, transpile
import numpy as np
import qiskit.quantum_info as qi

#hadamard gate
qreg_q = QuantumRegister(1, 'q')
circuit = QuantumCircuit(qreg_q)

circuit.h(0)

#transpile into new basis
print(circuit)
trans_qc = transpile(circuit, backend = None, basis_gates = ['rx', 'ry', 'rz', 'rxx'])

print(trans_qc)
print(qi.Operator(trans_qc))

   ┌───┐
q: ┤ H ├
   └───┘
global phase: π/2
   ┌─────────┐┌───────┐
q: ┤ Ry(π/2) ├┤ Rx(π) ├
   └─────────┘└───────┘
Operator([[ 0.70710678+0.00000000e+00j,  0.70710678-8.65956056e-17j],
          [ 0.70710678+0.00000000e+00j, -0.70710678+8.65956056e-17j]],
         input_dims=(2,), output_dims=(2,))


Cnot

In [27]:
#cnot gate
qreg_q = QuantumRegister(2, 'q')
circuit = QuantumCircuit(qreg_q)

circuit.cnot(0,1)

#transpile into new basis
print(circuit)
trans_qc = transpile(circuit, backend = None, basis_gates = ['rx', 'ry', 'rz', 'rxx'])

print(trans_qc)
print(qi.Operator(trans_qc))

          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
global phase: 7π/4
     ┌─────────┐┌───────────┐┌──────────┐┌──────────┐
q_0: ┤ Ry(π/2) ├┤0          ├┤ Rx(-π/2) ├┤ Ry(-π/2) ├
     └─────────┘│  Rxx(π/2) │├──────────┤└──────────┘
q_1: ───────────┤1          ├┤ Rx(-π/2) ├────────────
                └───────────┘└──────────┘            
Operator([[1.-1.11022302e-16j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
           0.+0.00000000e+00j],
          [0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
           1.-1.11022302e-16j],
          [0.+0.00000000e+00j, 0.+0.00000000e+00j, 1.-1.11022302e-16j,
           0.+0.00000000e+00j],
          [0.+0.00000000e+00j, 1.-1.11022302e-16j, 0.+0.00000000e+00j,
           0.+0.00000000e+00j]],
         input_dims=(2, 2), output_dims=(2, 2))


In [28]:
#cnot gate
qreg_q = QuantumRegister(2, 'q')
circuit = QuantumCircuit(qreg_q)

circuit.swap(0,1)

#transpile into new basis
print(circuit)
trans_qc = transpile(circuit, backend = None, basis_gates = ['rx', 'ry', 'rz', 'rxx'])

print(trans_qc)
print(qi.Operator(trans_qc))

#essentially classic 3 cnot decomposition

        
q_0: ─X─
      │ 
q_1: ─X─
        
global phase: 5π/4
     ┌─────────┐┌───────────┐┌──────────┐┌──────────┐┌───────────┐┌──────────┐»
q_0: ┤ Ry(π/2) ├┤0          ├┤ Rx(-π/2) ├┤ Ry(-π/2) ├┤1          ├┤ Rx(-π/2) ├»
     └─────────┘│  Rxx(π/2) │├──────────┤├─────────┬┘│  Rxx(π/2) │├──────────┤»
q_1: ───────────┤1          ├┤ Rx(-π/2) ├┤ Ry(π/2) ├─┤0          ├┤ Rx(-π/2) ├»
                └───────────┘└──────────┘└─────────┘ └───────────┘└──────────┘»
«     ┌─────────┐ ┌───────────┐┌──────────┐┌──────────┐
«q_0: ┤ Ry(π/2) ├─┤0          ├┤ Rx(-π/2) ├┤ Ry(-π/2) ├
«     ├─────────┴┐│  Rxx(π/2) │├──────────┤└──────────┘
«q_1: ┤ Ry(-π/2) ├┤1          ├┤ Rx(-π/2) ├────────────
«     └──────────┘└───────────┘└──────────┘            
Operator([[1.+6.66133815e-16j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
           0.+0.00000000e+00j],
          [0.+0.00000000e+00j, 0.+0.00000000e+00j, 1.+6.66133815e-16j,
           0.+0.00000000e+00j],
          [0.+0.00000000e+00j, 1.+6.66133815e-16j,